In [ ]:
# PART 1: Read data

import mne
import numpy as np
import scipy.io
from tqdm import tqdm
import os
from mne_icalabel import label_components
from mne.preprocessing import ICA
import re

# SEED_IV DATASET
sfreq = 200
n_sub = 15
volt_factor = 1e-6
channel_names = [
    "FP1", "FPZ", "FP2", 
    "AF3", "AF4", 
    "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8", 
    "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8", 
    "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8", 
    "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8", 
    "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8", 
    "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8", 
    "CB1", "O1", "OZ", "O2", "CB2"
]

# Select eeg channels
eeg_channel_names = [
    "FP1", "FPZ", "FP2", 
    "AF3", "AF4", 
    "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8", 
    "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8", 
    "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8", 
    "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8", 
    "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8", 
    "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8", 
    "CB1", "O1", "OZ", "O2", "CB2"
]
new_rate = 125
thresholds = [
    (3, 0.4),
    (30, 0.01)
]
montage_file_path = 'Z:/qingzhu/EEG_raw/SEED-VII/src/channel_62_pos.locs'
montage = mne.channels.read_custom_montage(montage_file_path)
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def detect_bad_channels(raw, thresholds):
    data = raw.get_data(picks='eeg')  # 获取 EEG 数据，形状为 (n_channels, n_times)
    sfreq = raw.info['sfreq']         # 采样频率
    total_samples = data.shape[1]     # 总采样点数
    bad_channels = set()              # 使用集合存储坏道以避免重复
    for a, b in thresholds:
        for ch_idx, ch_data in enumerate(data):
            median = np.median(np.abs(ch_data))
            high_values = np.abs(ch_data) > (a * median)
            high_ratio = np.sum(high_values) / total_samples
            if high_ratio > b:
                bad_channels.add(raw.info['ch_names'][ch_idx])
    return list(bad_channels)

def ICA_denoise(eeg):
    eeg = eeg.set_eeg_reference("average")
    n_components = np.floor(len(eeg.ch_names) * 0.78).astype(int)
    ica = ICA(n_components=n_components, max_iter="auto", random_state=42, method='infomax', fit_params=dict(extended=True))
    print(f'num_of_nan: {np.isnan(eeg.get_data()).sum()}')
    ica.fit(eeg)
    eeg.load_data()
    ic_labels = label_components(eeg, ica, method="iclabel")
    labels = ic_labels["labels"]
    exclude_idx = [
        idx for idx, label in enumerate(labels) if label not in ["brain", "other"]
    ]
    eeg_ica = eeg.copy()
    ica.apply(eeg_ica, exclude=exclude_idx)
    return eeg_ica

raws = [[]] * n_sub
eeg_selected = [[]] * n_sub
eeg_filted = [[]] * n_sub
eeg_interpolated = [[]] * n_sub
eeg_denoised = [[]] * n_sub
eeg_interpolated_2nd = [[]] * n_sub
debug = 1
save_dir = 'Z:/qingzhu/AutoICA_Processed_EEG/SEED-IV-new'


for sub in tqdm(range(n_sub), desc=f'Reading sub:'):
    raws_sub = []
    trial_id = 0
    for session_id in range(3):
        data_path = f'C:/Emotion_data/SEED_IV/eeg_raw_data/{session_id+1}/{sub+1}.mat'
        mat_dic = scipy.io.loadmat(data_path)
        filtered_dict = {key: value for key, value in mat_dic.items() if 'eeg' in key}
        sorted_items = sorted(filtered_dict.items(), key=lambda x: natural_sort_key(x[0]))  # 按键排序
        result_list = [value for key, value in sorted_items]
        info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types='eeg')
        for _ in range(len(result_list)):
            eeg_trial_i = result_list[trial_id]
            try:
                raw_i = mne.io.RawArray(eeg_trial_i * volt_factor, info)
                if debug:
                    raws[sub].append(raw_i)
            except:
                print(f'ERROR Sub {sub+1} Trial {trial_id}')
                if debug:
                    raws[sub].append(None)
                continue

            # selecting channel
            print('Selecting Channel...')
            try:
                raw_selected = raw_i.copy().pick_channels(eeg_channel_names)
                if debug:
                    eeg_selected[sub].append(raw_selected)
            except:
                if debug:
                    eeg_selected[sub].append(None)

            # Filter & downsample eeg data
            print('Filter & Downsample...')
            try:
                trial_i_downsampled = raw_selected.resample(sfreq=new_rate)
                trial_i_filted = trial_i_downsampled.filter(1, 47, fir_design='firwin')
                if debug:
                    eeg_filted[sub].append(trial_i_filted)
            except:
                if debug:
                    eeg_filted[sub].append(None)

            # 1st bad channel interpolation eeg data
            print('1st bad channel interpolation...')
            try:
                trial_i_interploated = trial_i_filted.copy()
                bad_channels = detect_bad_channels(trial_i_filted, thresholds)
                print(f"Detected bad channels: {bad_channels}")
                trial_i_interploated.info['bads'] = bad_channels
                trial_i_interploated.set_montage(montage)
                trial_i_interploated.interpolate_bads(reset_bads=True, exclude=['Fp1', 'Fp2', 'F7', 'F8', 'AF7', 'AF8'])
                if debug:
                    eeg_interpolated[sub].append(trial_i_interploated)
            except:
                if debug:
                    eeg_interpolated[sub].append(None)

            # ICA denoise
            print('ICA denoise...')
            try:
                trial_i_ica = trial_i_interploated.copy()
                trial_i_ica = ICA_denoise(trial_i_ica)
                if debug:
                    eeg_denoised[sub].append(trial_i_ica)
            except:
                if debug:
                    eeg_denoised[sub].append(None)

            # 2nd bad channel interploation
            print('2nd bad channel interploation...')
            try:
                trial_i_interpolated = trial_i_ica.copy()
                bad_channels = detect_bad_channels(trial_i_interpolated, thresholds)
                print(f"Detected bad channels: {bad_channels}")
                trial_i_interpolated.info['bads'] = bad_channels
                trial_i_interploated.set_montage(montage)
                trial_i_interpolated.interpolate_bads(reset_bads=True)
                trial_i_interpolated.set_eeg_reference("average")
                if debug:
                    eeg_interpolated_2nd[sub].append(trial_i_interpolated)
            except:
                if debug:
                    eeg_interpolated_2nd[sub].append(None)

            # Save data
            print('Save data...')
            sub_dir = os.path.join(save_dir, f'sub_{sub}')
            if not os.path.exists(sub_dir):  # 如果子路径不存在，创建它
                os.makedirs(sub_dir)
            try:
                data_dir = os.path.join(sub_dir, f'eeg_sub_{sub}_trial_{trial_id}')
                np.save(data_dir, trial_i_interpolated.get_data())
            except:
                continue
            
            trial_id += 1
            

Reading sub::   0%|          | 0/15 [00:00<?, ?it/s]

Creating RawArray with float64 data, n_channels=62, n_times=33601
    Range : 0 ... 33600 =      0.000 ...   168.000 secs
Ready.
Selecting Channel...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filter & Downsample...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 47 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 47.00 Hz
- Upper transition bandwidth: 11.75 Hz (-6 dB cutoff frequency: 52.88 Hz)
- Filter length: 413 samples (3.304 s)

1st bad channel interpolation...
Detected bad channels: []
Setting channel interpolation method to {'eeg': 'spline'}.
ICA denoise...
EEG channel type selected for re-referencing
Applying

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
C:\Users\ncclab\AppData\Local\Temp\ipykernel_53020\3873607666.py:140: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  trial_i_interploated.interpolate_bads(reset_bads=True, exclude=['Fp1', 'Fp2', 'F7', 'F8', 'AF7', 'AF8'])


Selecting by number: 48 components
Computing Extended Infomax ICA
Fitting ICA took 11.0s.


C:\Users\ncclab\AppData\Local\Temp\ipykernel_53020\3873607666.py:71: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(eeg, ica, method="iclabel")


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
C:\Users\ncclab\AppData\Local\Temp\ipykernel_53020\3873607666.py:140: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  trial_i_interploated.interpolate_bads(reset_bads=True, exclude=['Fp1', 'Fp2', 'F7', 'F8', 'AF7', 'AF8'])


Selecting by number: 48 components
Computing Extended Infomax ICA
Fitting ICA took 192.8s.


C:\Users\ncclab\AppData\Local\Temp\ipykernel_53020\3873607666.py:71: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(eeg, ica, method="iclabel")


Applying ICA to Raw instance
    Transforming to ICA space (48 components)
    Zeroing out 18 ICA components
    Projecting back using 62 PCA components
2nd bad channel interploation...
Detected bad channels: []
Setting channel interpolation method to {'eeg': 'spline'}.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Save data...


C:\Users\ncclab\AppData\Local\Temp\ipykernel_53020\3873607666.py:166: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  trial_i_interpolated.interpolate_bads(reset_bads=True)
Reading sub::  60%|██████    | 9/15 [6:57:52<4:38:34, 2785.78s/it]


In [1]:
raws

NameError: name 'raws' is not defined